# YOLOX

In [ ]:
import onnx
from io import BytesIO
import cv2
import urllib
import numpy as np
import onnxruntime
from PIL import Image
import matplotlib.pyplot as plt
import tritonclient.grpc as grpcclient
import os

TRITON_PORT_GRPC = os.getenv("TRITON_PORT_GRPC", "8001")
MLFLOW_PORT = os.getenv("MLFLOW_PORT", "8080")

def preprocess(img, input_size=(640,640), swap=(2, 0, 1)):
    if len(img.shape) == 3:
        padded_img = np.ones((input_size[0], input_size[1], 3), dtype=np.uint8) * 114
    else:
        padded_img = np.ones(input_size, dtype=np.uint8) * 114

    r = min(input_size[0] / img.shape[0], input_size[1] / img.shape[1])
    resized_img = cv2.resize(
        img,
        (int(img.shape[1] * r), int(img.shape[0] * r)),
        interpolation=cv2.INTER_LINEAR,
    ).astype(np.uint8)
    padded_img[: int(img.shape[0] * r), : int(img.shape[1] * r)] = resized_img

    padded_img = padded_img.transpose(swap)
    padded_img = np.ascontiguousarray(padded_img, dtype=np.float32)
    return padded_img, r

def postprocess(outputs, input_size=(640,640), p6=False):
    grids = []
    expanded_strides = []
    strides = [8, 16, 32] if not p6 else [8, 16, 32, 64]

    hsizes = [input_size[0] // stride for stride in strides]
    wsizes = [input_size[1] // stride for stride in strides]

    for hsize, wsize, stride in zip(hsizes, wsizes, strides):
        xv, yv = np.meshgrid(np.arange(wsize), np.arange(hsize))
        grid = np.stack((xv, yv), 2).reshape(1, -1, 2)
        grids.append(grid)
        shape = grid.shape[:2]
        expanded_strides.append(np.full((*shape, 1), stride))

    grids = np.concatenate(grids, 1)
    expanded_strides = np.concatenate(expanded_strides, 1)
    outputs[..., :2] = (outputs[..., :2] + grids) * expanded_strides
    outputs[..., 2:4] = np.exp(outputs[..., 2:4]) * expanded_strides

    return outputs


# Load YOLOX model

In [ ]:
ONNX_MODEL_NAME = "yolox_s"

req = urllib.request.urlopen(f"https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/{ONNX_MODEL_NAME}.onnx")
model_data = BytesIO(req.read())
model = onnx.load(model_data)
onnx.checker.check_model(model)
# onnx.save_model(model, "yolox_s_save.onnx")

# Register model to mlflow

In [ ]:
import mlflow
from mlflow.models import infer_signature

mlflow.set_tracking_uri(f"http://localhost:{MLFLOW_PORT}")

mlflow.set_experiment(ONNX_MODEL_NAME)

with mlflow.start_run():
    mlflow_model = mlflow.onnx.log_model(model, ONNX_MODEL_NAME, registered_model_name=ONNX_MODEL_NAME, save_as_external_data=False)

# Deploy model to Triton

In [ ]:
from mlflow import MlflowClient
from mlflow.deployments import get_deploy_client

# retrieve the most recent model version
mlflow_client = MlflowClient()
model_version = max(m.version for m in mlflow_client.search_model_versions(f"name='{ONNX_MODEL_NAME}'"))

client = get_deploy_client('triton')
client.create_deployment(ONNX_MODEL_NAME, f"models:/{ONNX_MODEL_NAME}/{model_version}", flavor="onnx")

# Load image

In [ ]:
# preprocess image
IMAGE_URL = "https://github.com/ultralytics/ultralytics/blob/main/ultralytics/assets/bus.jpg?raw=true"
req = urllib.request.urlopen(IMAGE_URL)
origin_img = np.asarray(Image.open(BytesIO(req.read()))).astype(np.uint8)
img, ratio = preprocess(origin_img[:,:,:3])

plt.axis('off')
plt.imshow(origin_img)

# Run prediction

In [ ]:
COCO_CLASSES=("person","bicycle","car","motorcycle","airplane","bus","train","truck","boat","trafficlight","firehydrant","stopsign","parkingmeter","bench","bird","cat","dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack","umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sportsball","kite","baseballbat","baseballglove","skateboard","surfboard","tennisracket","bottle","wineglass","cup","fork","knife","spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot","hotdog","pizza","donut","cake","chair","couch","pottedplant","bed","diningtable","toilet","tv","laptop","mouse","remote","keyboard","cellphone","microwave","oven","toaster","sink","refrigerator","book","clock","vase","scissors","teddybear","hairdrier","toothbrush",)
COLORS=np.array([0.000,0.447,0.741,0.850,0.325,0.098,0.929,0.694,0.125,0.494,0.184,0.556,0.466,0.674,0.188,0.301,0.745,0.933,0.635,0.078,0.184,0.300,0.300,0.300,0.600,0.600,0.600,1.000,0.000,0.000,1.000,0.500,0.000,0.749,0.749,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.667,0.000,1.000,0.333,0.333,0.000,0.333,0.667,0.000,0.333,1.000,0.000,0.667,0.333,0.000,0.667,0.667,0.000,0.667,1.000,0.000,1.000,0.333,0.000,1.000,0.667,0.000,1.000,1.000,0.000,0.000,0.333,0.500,0.000,0.667,0.500,0.000,1.000,0.500,0.333,0.000,0.500,0.333,0.333,0.500,0.333,0.667,0.500,0.333,1.000,0.500,0.667,0.000,0.500,0.667,0.333,0.500,0.667,0.667,0.500,0.667,1.000,0.500,1.000,0.000,0.500,1.000,0.333,0.500,1.000,0.667,0.500,1.000,1.000,0.500,0.000,0.333,1.000,0.000,0.667,1.000,0.000,1.000,1.000,0.333,0.000,1.000,0.333,0.333,1.000,0.333,0.667,1.000,0.333,1.000,1.000,0.667,0.000,1.000,0.667,0.333,1.000,0.667,0.667,1.000,0.667,1.000,1.000,1.000,0.000,1.000,1.000,0.333,1.000,1.000,0.667,1.000,0.333,0.000,0.000,0.500,0.000,0.000,0.667,0.000,0.000,0.833,0.000,0.000,1.000,0.000,0.000,0.000,0.167,0.000,0.000,0.333,0.000,0.000,0.500,0.000,0.000,0.667,0.000,0.000,0.833,0.000,0.000,1.000,0.000,0.000,0.000,0.167,0.000,0.000,0.333,0.000,0.000,0.500,0.000,0.000,0.667,0.000,0.000,0.833,0.000,0.000,1.000,0.000,0.000,0.000,0.143,0.143,0.143,0.286,0.286,0.286,0.429,0.429,0.429,0.571,0.571,0.571,0.714,0.714,0.714,0.857,0.857,0.857,0.000,0.447,0.741,0.314,0.717,0.741,0.50,0.5,0]).astype(np.float32).reshape(-1,3)

def multiclass_nms_class_agnostic(boxes, scores, nms_thr, score_thr):
    """Multiclass NMS implemented in Numpy. Class-agnostic version."""
    cls_inds = scores.argmax(1)
    cls_scores = scores[np.arange(len(cls_inds)), cls_inds]

    valid_score_mask = cls_scores > score_thr
    if valid_score_mask.sum() == 0:
        return None
    valid_scores = cls_scores[valid_score_mask]
    valid_boxes = boxes[valid_score_mask]
    valid_cls_inds = cls_inds[valid_score_mask]
    keep = nms(valid_boxes, valid_scores, nms_thr)
    if keep:
        dets = np.concatenate(
            [valid_boxes[keep], valid_scores[keep, None], valid_cls_inds[keep, None]], 1
        )
    return dets

def nms(boxes, scores, nms_thr):
    """Single class NMS implemented in Numpy."""
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= nms_thr)[0]
        order = order[inds + 1]

    return keep


def visualize(img, boxes, scores, cls_ids, conf=0.5, class_names=None):

    for i in range(len(boxes)):
        box = boxes[i]
        cls_id = int(cls_ids[i])
        score = scores[i]
        if score < conf:
            continue
        x0 = int(box[0])
        y0 = int(box[1])
        x1 = int(box[2])
        y1 = int(box[3])

        color = (COLORS[cls_id] * 255).astype(np.uint8).tolist()
        text = '{}:{:.1f}%'.format(class_names[cls_id], score * 100)
        txt_color = (0, 0, 0) if np.mean(COLORS[cls_id]) > 0.5 else (255, 255, 255)
        font = cv2.FONT_HERSHEY_SIMPLEX

        txt_size = cv2.getTextSize(text, font, 0.4, 1)[0]
        cv2.rectangle(img, (x0, y0), (x1, y1), color, 2)

        txt_bk_color = (COLORS[cls_id] * 255 * 0.7).astype(np.uint8).tolist()
        cv2.rectangle(
            img,
            (x0, y0 + 1),
            (x0 + txt_size[0] + 1, y0 + int(1.5*txt_size[1])),
            txt_bk_color,
            -1
        )
        cv2.putText(img, text, (x0, y0 + txt_size[1]), font, 0.4, txt_color, thickness=1)

    return img


# create triton client
#  Note that the same model can be executed locally, using the following code snippet:
# 
# session = onnxruntime.InferenceSession(model.SerializeToString())
# ort_inputs = {session.get_inputs()[0].name: img[None, :, :, :]}
# output = session.run(None, ort_inputs)[0]

triton_client = grpcclient.InferenceServerClient(
    url=f"localhost:{TRITON_PORT_GRPC}", verbose=True
)
input = grpcclient.InferInput("images", [1,3,640,640], "FP32")
input.set_data_from_numpy(img[None,:,:,:])
result = triton_client.infer(ONNX_MODEL_NAME, inputs=[input])
output = np.copy(result.as_numpy("output"))

# postprocessing step - to extract bounding boxes

predictions = postprocess(output)[0]
boxes = predictions[:, :4]
scores = predictions[:, 4:5] * predictions[:, 5:]

boxes_xyxy = np.ones_like(boxes)
boxes_xyxy[:, 0] = boxes[:, 0] - boxes[:, 2]/2.
boxes_xyxy[:, 1] = boxes[:, 1] - boxes[:, 3]/2.
boxes_xyxy[:, 2] = boxes[:, 0] + boxes[:, 2]/2.
boxes_xyxy[:, 3] = boxes[:, 1] + boxes[:, 3]/2.
boxes_xyxy /= ratio

dets = multiclass_nms_class_agnostic(boxes_xyxy, scores, nms_thr=0.45, score_thr=0.1)
if dets is not None:
    final_boxes, final_scores, final_cls_inds = dets[:, :4], dets[:, 4], dets[:, 5]
    origin_img = visualize(origin_img, final_boxes, final_scores, final_cls_inds,
                         conf=0.3, class_names=COCO_CLASSES)
    plt.figure(figsize = (200,20))
    plt.axis('off')
    plt.imshow(origin_img)